In [1]:
# baseline cnn model for digit
import numpy as np
#import import_ipynb
from loader import Loader
from sklearn.model_selection import KFold
from keras.utils import to_categorical
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import BatchNormalization
import random as r
import matplotlib.pyplot as plt
from keras.preprocessing.image import img_to_array, load_img
from matplotlib import pyplot
import scipy.stats as stats
import os
from keras.utils.vis_utils import plot_model


In [2]:
# function that returns the data in the required format

def dataset1():
# Dividing the data into train and test data
    #clear = lambda: os.system('clear')
    #clear()
    
    dataset = Loader()
    noise = 1
    print('Noise intensity is')
    print(noise)
    
    trainX,trainY =  dataset.getWholeTrainSet(pca=False) 
    
    
    trainXaug,trainYaug =  dataset.augment() 

    trainXwithnoise, trainYwithnoise = dataset.getNoisySet(intensity=noise, set="train", flat=False, truncate=True)
    
        
    trainXaug1 = np.concatenate((trainX, trainXaug))
    trainYaug1 = np.concatenate((trainY, trainY))
   
    testX, testY = dataset.getWholeTestSet()
    
    trainX = np.array(trainX)
    trainXwithnoise = np.array(trainXwithnoise)
    trainY = np.array(trainY)
    testX = np.array(testX)
    testY = np.array(testY)
   
    trainXwithnoise = trainXwithnoise.reshape((trainXwithnoise.shape[0], 16,15, 1))
    trainXaug1 =  trainXaug1.reshape((trainXaug1.shape[0], 16,15, 1))
    testX = testX.reshape((testX.shape[0], 16,15, 1))
    trainX = trainX.reshape((trainX.shape[0], 16,15, 1))
    # the labels are one hot encoded
    testYintegers = testY
    trainYaug1 =  to_categorical(trainYaug1)
    testY = to_categorical(testY)
    trainY = to_categorical(trainY)
    return trainXaug1,trainXwithnoise, trainYaug1, testX, testY,testYintegers

In [3]:
def getdataset1(intensity,dataset):
    
    trainX,trainXwithnoise, trainY, testX, testY,testYintegers = dataset1(intensity,dataset)
    return trainX,trainXwithnoise, trainY, testX, testY,testYintegers

In [4]:
# CNN model
def model_definition():
    # tune model parameters here
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(16,15, 1)))
    #model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    
    
    
    # Flatten and apply drop out or apply drop out after Conv2D it is essentially the same
    model.add(Dropout(0.50))
    model.add(Dense(240, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    
    
    model.add(Dropout(0.20))
    model.add(Dense(120, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    
    model.add(Dropout(0.20))
    model.add(Dense(50, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    
    
    
    
#    model.add(Dropout(0.20))
    model.add(Dense(10, activation='softmax'))
    #opt = SGD(lr=0.01)
    opt = SGD(lr=0.03, momentum=0.9)
    #opt = 'adam'
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
# kfold cross validation
def train_modelkfold(mtrainX,mtrainXwithnoise, mtrainY, folds=10):
    accuracies,histories = list(), list()

    kfold = KFold(folds, shuffle=True,random_state=3)      
    for training_indices, testing_indices in kfold.split(mtrainX):
        model = model_definition()
        trainX, trainY, testX, testY = mtrainX[training_indices], mtrainY[training_indices], mtrainX[testing_indices], mtrainY[testing_indices]
        # so this model.fit reflects the model.fit for only the kfold 
        #model.fit(trainX, trainY, epochs=120, batch_size=10, validation_data=(testX, testY), verbose=0)
        history_kfold = model.fit(trainX, trainY, epochs=200, batch_size=32, validation_data=(testX, testY), verbose=0)
        
        # returning the validation accuracies of every k fold validation set
        _, accuraccy = model.evaluate(testX, testY, verbose=0)
        print('%.3f' % (accuraccy * 100.0))
        # stores accuracies and test data on last kth model
        accuracies.append(accuraccy)
        histories.append(history_kfold)
        # implement a list for storing all the k fold models. If you wish
    return accuracies,histories

In [8]:
# train and return models 
def fitmodel(trainX1, trainY,testX, testY,i):   
    
    #once the model has been decided on from k fold
    model = model_definition()
    history_training = model.fit(trainX1, trainY,epochs=200, verbose=0)
    # save model
    name = "final_model.h5" + str(i)
    model.save(name)
    # load the model
    model = load_model(name)
    return model,history_training

In [9]:

# Counts the total number of correctly predicted labels in the test data

def modelpredictions(modellist,testX,testY):
    
    
    correctpernoise = []
    for model in modellist:
        correctaveragepermodel = 0.0
        correct = 0.0
        preds = model.predict(testX)
        preds = np.argmax(model.predict(testX), axis=-1)
        #preds = to_categorical(preds)
        #print(preds)
        
        for i in range(len(testY)):
            if (preds[i] == testY[i]): # Change
                correct = correct+1
                
        correctaveragepermodel = correct/len(testY)
        correctpernoise.append(correctaveragepermodel)
        #return list
    return correctpernoise

In [10]:
# final function to gather the average per model

def results():
    
    trainingaccuracypermodel  = []
    testaccuracypernoisemean = []
    testaccuracyvariancepernoise = []    
    numofmodels = 10

    dataset = Loader()
    traningmean = []
    trainingvariance = []
    traininglosspernoiseintensity = []
    
    
    kfoldmeanpermodel = []
    kfoldvariancepermodel = []
    histories = list()
    modellist = []

    trainX,trainXwithnoise, trainY, testX, testY,testYintegers = dataset1()
    print('printing the k fold accuracies with noisy training data')
    accuracies, histories = train_modelkfold(trainX,trainXwithnoise, trainY)
    print('mean of k-fold accuracies')
    print(np.mean(accuracies))
    kfoldmeanpermodel.append(np.mean(accuracies))
    print('standard of k-fold accuracies')
    print(np.std(accuracies))
    kfoldvariancepermodel.append(np.std(accuracies))
    print('finished k fold')

    trainiaccuracy = 0.0
    testmean = 0.0
    testvariance = 0.0
    errorresultpernoise=0.0
    trainloss = 0.0

    # making  models
    for i in range(numofmodels):
        trainvariance = 0.0
        model,history_training = fitmodel(trainX, trainY,testX, testY,i)
        histories.append(history_training)
        modellist.append(model)
        print('In training')  
        #print(type(trainiaccuracy))
        trainingaccuracypermodel.append(sum(histories[i].history['accuracy'])/200)


        #training mean
        traningmean.append(np.mean(trainingaccuracypermodel))
        
        # training variance
        trainingvariance.append(np.std(trainingaccuracypermodel))

        
        #correct = 0    
        print('In testing')
        correctpermodel = modelpredictions(modellist,testX,testYintegers)
        
       
        #testmean = (sum(correctpermodel)/numofmodels)*100
        
        # test mean
        testmean = np.mean(correctpermodel)
        testaccuracypernoisemean.append(testmean)
        
        #standard deviation of test accuracy per noise
        testvariance = np.std(correctpermodel)
        testaccuracyvariancepernoise.append(testvariance)    
    
    
        %reset -f in
    return testaccuracypernoisemean,testaccuracyvariancepernoise,traningmean,trainingvariance,kfoldmeanpermodel,kfoldvariancepermodel
    #predslist.append(preds)

In [11]:
testaccuracypernoisemean,testaccuracyvariancepernoise,traningmean,trainingvariance,kfoldmeanpermodel,kfoldvariancepermodel = results()

Noise intensity is
1
printing the k fold accuracies with noisy training data
99.500
100.000
99.750
99.500
99.750
100.000
99.750
100.000
99.750
99.500
mean of k-fold accuracies
0.9975000023841858
standard of k-fold accuracies
0.0019364898263213358
finished k fold
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: final_model.h50/assets
In training
In testing
Flushing input history
INFO:tensorflow:Assets written to: final_model.h51/assets
In training
In testing
Flushing input history
INFO:tensorflow:Assets written to: final_model.h52/assets
In training
In testing
Flushing input history
INFO:tensorflow:Assets written to: final_model.h53/assets
In training
In testing
Flushing input history
INFO:tensorflow:Assets written to: final_model.h54/assets
In training
In testing
Flushi

In [12]:
testaccuracypernoisemean

[0.984,
 0.9844999999999999,
 0.983,
 0.9834999999999999,
 0.9825999999999999,
 0.9818333333333332,
 0.982142857142857,
 0.98225,
 0.9824444444444443,
 0.9827999999999999]

In [18]:
print(np.mean(testaccuracypernoisemean))

0.9829070634920634


In [13]:
testaccuracyvariancepernoise

[0.0,
 0.0005000000000000004,
 0.0021602468994692888,
 0.0020615528128088323,
 0.0025768197453450276,
 0.0029107081994288325,
 0.0027994168488950635,
 0.002633913438213187,
 0.0025434495871688016,
 0.002638181191654586]

In [14]:
traningmean

[0.9952333354949952,
 0.9952708354592323,
 0.9954166689515113,
 0.9954746552556752,
 0.9954894473552702,
 0.9954682898024716,
 0.9954593280383517,
 0.9954406278207898,
 0.9954574103156726,
 0.995430141746998]

In [15]:
trainingvariance

[0.0,
 3.749996423718649e-05,
 0.00020850015144469053,
 0.0002066191406334811,
 0.00018715875684192703,
 0.00017728097403360468,
 0.00016559176032112665,
 0.00016260670084146565,
 0.00016048761355859546,
 0.00017283755446450202]

In [16]:
kfoldmeanpermodel

[0.9975000023841858]

In [17]:
kfoldvariancepermodel

[0.0019364898263213358]